# Creating a Batch Inferencing Service

In previous labs, you used an Azure ML *pipeline* to automate the training and registration of a model, and you published a model as a web service for real-time *inferencing* (getting predictions from a model). Now you'll combine these two concepts to create a pipeline for *batch inferencing*. What does that mean? Well, imagine a health clinic takes patient measurements all day, saving the details for each patient in a separate file. Then overnight, the diabetes prediction model can be used to process all of the day's patient data as a batch, generating predictions that will be waiting the following morning so that the clinic can follow up with patients who are predicted to be at risk of diabetes. That's what you'll implement in this exercise.

Let's start by ensuring that you have the latest version of the Azure ML SDK installed.

In [1]:
!pip install --upgrade azureml-sdk[notebooks,automl,explain]

Requirement already up-to-date: azureml-sdk[automl,explain,notebooks] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.5.0)


## Connect to Your Workspace

Now connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.5.0 to work with azure-ml-demo


## Train and Register a Model

If you have completed the previous lab in this course, you will have registered many versions of the **diabetes_model** model, and you're ready to go.

If you haven't, you can run the cell below to train and register the model.

In [3]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Starting experiment: diabetes-training
Loading Data...
Training a decision tree model
Accuracy: 0.891
AUC: 0.8776133140952285
Model trained and registered.


## Generate and Upload Batch Data

Since we don't actually have a fully staffed clinic with patients from whom to get new data for this course, you'll generate a random sample from our diabetes CSV file and use those to test the pipeline. Then you'll upload that data to a datastore in the Azure Machine Learning workspace and register a dataset for it.

In [4]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os

# Load the diabetes data
diabetes = pd.read_csv('data/diabetes2.csv')
# Get a 100-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=100).values

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")

# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-data',
                                             description='batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Folder created!
Saving files...
files saved!
Uploading files to datastore...
Uploading an estimated of 100 files
Uploading batch-data/1.csv
Uploading batch-data/10.csv
Uploading batch-data/100.csv
Uploading batch-data/11.csv
Uploading batch-data/12.csv
Uploading batch-data/13.csv
Uploading batch-data/14.csv
Uploading batch-data/15.csv
Uploading batch-data/16.csv
Uploading batch-data/17.csv
Uploading batch-data/18.csv
Uploading batch-data/19.csv
Uploading batch-data/2.csv
Uploading batch-data/20.csv
Uploading batch-data/21.csv
Uploading batch-data/22.csv
Uploading batch-data/23.csv
Uploading batch-data/24.csv
Uploading batch-data/25.csv
Uploading batch-data/26.csv
Uploading batch-data/27.csv
Uploading batch-data/28.csv
Uploading batch-data/29.csv
Uploading batch-data/3.csv
Uploading batch-data/30.csv
Uploading batch-data/32.csv
Uploading batch-data/33.csv
Uploading batch-data/34.csv
Uploading batch-data/35.csv
Uploading batch-data/31.csv
Uploading batch-data/36.csv
Uploading batch-data/

## Create Compute

We'll need a compute context for the pipeline, so we'll use the Azure ML compute cluster you used in the previous exercises (it will be created if it doesn't already exist).

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# Check for existing cluster
try:
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create an AzureMl Compute resource
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

inference_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Create a Pipeline for Batch Inferencing

Now we're ready to define the pipeline we'll use for batch inferencing. Our pipeline will need Python code to perform the batch inferencing, so let's create a folder where we can keep all the files used by the pipeline:

In [6]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

batch_pipeline


Now we'll create a Python script to do the actual work, and save it in the pipeline folder:

In [7]:
%%writefile $experiment_folder/batch_diabetes.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read the comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for prediction (model expects multiple items)
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Writing batch_pipeline/batch_diabetes.py


Next we'll define a run context that includes the dependencies required by the script

In [8]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import CondaDependencies

# Add dependencies required by the model
# For scikit-learn models, you need scikit-learn
cd = CondaDependencies.create(pip_packages=['scikit-learn'])

batch_env = Environment(name='batch_environment')
batch_env.python.conda_dependencies = cd
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


You're going to use a pipeline to run the batch prediction script, generate predictions from the input data, and save the results as a text file in the output folder. To do this, you can use a **ParallelRunStep**, which enables the batch data to be processed in parallel and the results collated in a single output file named *parallel_run_step.txt*.

So you'll need to install the library containing the **ParallelRunStep** class.

In [9]:
!pip install --upgrade azureml-contrib-pipeline-steps

  Found existing installation: azureml-contrib-pipeline-steps 1.4.0
    Uninstalling azureml-contrib-pipeline-steps-1.4.0:
      Successfully uninstalled azureml-contrib-pipeline-steps-1.4.0


OK, now you're ready to define a **ParallelRunStep** pipeline step.

In [10]:
from azureml.contrib.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData

default_ds = ws.get_default_datastore()
model = ws.models['diabetes_model']

output_dir = PipelineData(name='inferences', 
                          datastore=default_ds, 
                          output_path_on_compute='diabetes/results')

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="batch_diabetes.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-diabetes',
    models=[model],
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('diabetes_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


Now it's time to put the step into a pipeline, and run it.

> **Note**: This may take some time!

In [11]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'batch_prediction_pipeline').submit(pipeline)
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Created step batch-score-diabetes [2c354f3f][b84cd788-a8d4-48ac-865c-c8d627e34ece], (This step will run and generate new outputs)
Created data reference diabetes_batch_0 for StepId [a8e3ed31][0d7276cb-b1d2-4ed6-bb18-db49d7736760], (Consumers of this data will generate new runs.)
Submitted PipelineRun fc31d832-3ca9-49b9-ad8f-bf20d5fcdd4c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_prediction_pipeline/runs/fc31d832-3ca9-49b9-ad8f-bf20d5fcdd4c?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: fc31d832-3ca9-49b9-ad8f-bf20d5fcdd4c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_prediction_pipeline/runs/fc31d832-3ca9-49b9-ad8f-bf20d5fcdd4c?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 6f03aff6-f651-4a2b-a0fe-ce9ded78b952
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_prediction_pipeline/runs/6f03aff6-f651-4a2b-a0fe-ce9ded78b952?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo
StepRun( batch-score-diabetes ) Status: NotStarted
StepRun( batch-score-diabetes ) Status: Running

Streaming azureml-logs/20_image_build_log.txt
2020/05/18 13:53:03 Downloading source code...
2020/05/18 13:53:04 Finished downloading source code
2020/05/18 13:53:05 Creating Docker network: acb_default_network, driver: 'bridge'
2020/05/

'Finished'

When the pipeline has finished running, the resulting predictions will have been saved in the outputs of the experiment associated with the first (and only) step in the pipeline. You can retrieve it as follows:

In [12]:
import pandas as pd
import shutil

shutil.rmtree('diabetes-results', ignore_errors=True)

prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')


for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,18.csv,1
1,19.csv,0
2,2.csv,0
3,20.csv,1
4,21.csv,0
5,27.csv,0
6,28.csv,0
7,29.csv,1
8,3.csv,1
9,30.csv,1


## Publish the Pipeline and use its REST Interface

Now that you have a working pipeline for batch inferencing, you can publish it and use a REST endpoint to run it from an application.

In [13]:
published_pipeline = pipeline_run.publish_pipeline(
    name='Diabetes_Parallel_Batch_Pipeline', description='Batch scoring of diabetes data', version='1.0')

published_pipeline

Name,Id,Status,Endpoint
Diabetes_Parallel_Batch_Pipeline,eda8afac-26f2-416b-8867-6f1329e2fb5d,Active,REST Endpoint


Note that the published pipeline has an endpoint, which you can see in the Azure portal. You can also find it as a property of the published pipeline object:

In [14]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://eastus2.api.azureml.ms/pipelines/v1.0/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourceGroups/azure-ml-demo/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-demo/PipelineRuns/PipelineSubmit/eda8afac-26f2-416b-8867-6f1329e2fb5d


To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. To test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

> **Note**: A real application would require a service principal with which to be authenticated.

In [15]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

Authentication header ready.


Now we're ready to call the REST interface. The pipeline runs asynchronously, so we'll get an identifier back, which we can use to track the pipeline experiment as it runs:

In [16]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "Batch_Pipeline_via_REST"})
run_id = response.json()["Id"]
run_id

'442d7440-f012-4832-be67-f37ee11cf809'

Since we have the run ID, we can use the **RunDetails** widget to view the experiment as it runs:

In [17]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["Batch_Pipeline_via_REST"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

As before, the results are in the output of the first pipeline step:

In [19]:
import pandas as pd
import shutil

shutil.rmtree("diabetes-results", ignore_errors=True)

prediction_run = next(published_pipeline_run.get_children())
prediction_output = prediction_run.get_output_data("inferences")
prediction_output.download(local_path="diabetes-results")


for root, dirs, files in os.walk("diabetes-results"):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,18.csv,1
1,19.csv,0
2,2.csv,0
3,20.csv,1
4,21.csv,0
5,27.csv,0
6,28.csv,0
7,29.csv,1
8,3.csv,1
9,30.csv,1


Now you have a pipeline that can be used to batch process daily patient data.

**More Information**: For more details about using pipelines for batch inferencing, see the [How to Run Batch Predictions](https://docs.microsoft.com/azure/machine-learning/how-to-run-batch-predictions) in the Azure Machine Learning documentation.